In [1]:
## import dependencies

from mixpanel_api import Mixpanel
import pandas as pd
from pandas.io.json import json_normalize
import requests
import json
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)


In [2]:
## import data from MixPanel

if __name__ == '__main__':
  credentials = {
    'API_secret': '325f04b0482a5ce8b015ee5ea75e5f21',
    'token': '',
  }

  m = Mixpanel(credentials['API_secret'])
# # selector = ''  
#   selector = '(("US" in properties["$country_code"]) and (defined (properties["$country_code"])))'
#   parameters = {'selector': selector}

# m.export_people('people_export.txt',parameters,timezone_offset=-8)


In [3]:
## import txt file as json

with open('people_export.txt', 'r') as handle:
    parsed = json.load(handle)

## printing the json file, if needed    
#print(json.dumps(parsed, indent=4, sort_keys=True))



In [4]:
## convert json to dataframe

people_df = pd.DataFrame.from_dict(json_normalize(parsed), orient='columns')


In [10]:
#extract date and time from subs
people_df['Sub_End_Date'] = pd.to_datetime(people_df['$properties.$subscription_end_time']).dt.date
people_df['Sub_End_Time'] = pd.to_datetime(people_df['$properties.$subscription_end_time']).dt.time

In [11]:
people_df.head()

,$distinct_id,$properties.$ae_first_app_open_date,$properties.$ae_total_app_session_length,$properties.$ae_total_app_sessions,$properties.$android_app_version,$properties.$android_app_version_code,$properties.$android_brand,$properties.$android_devices,$properties.$android_lib_version,$properties.$android_manufacturer,$properties.$android_model,$properties.$android_os,$properties.$android_os_version,$properties.$android_push_error,$properties.$bounce_category,$properties.$bounce_notification,$properties.$bounced,$properties.$campaigns,$properties.$city,$properties.$country_code,$properties.$created_at,$properties.$distinct_id,$properties.$email,$properties.$first_name,$properties.$ios_app_release,$properties.$ios_app_version,$properties.$ios_device_model,$properties.$ios_devices,$properties.$ios_lib_version,$properties.$ios_version,$properties.$ip_address,$properties.$journeys,$properties.$last_activity_time,$properties.$last_name,$properties.$last_seen,$properties.$marked_spam,$properties.$mp_message_sends.c3601296.c,$properties.$mp_message_sends.c3601296.t,$properties.$mp_message_sends.c3601316.c,$properties.$mp_message_sends.c3601316.t,$properties.$mp_message_sends.c3601339.c,$properties.$mp_message_sends.c3601339.t,$properties.$mp_message_sends.c3601351.c,$properties.$mp_message_sends.c3601351.t,$properties.$mp_message_sends.c3610479.c,$properties.$mp_message_sends.c3610479.t,$properties.$mp_message_sends.c3614068.c,$properties.$mp_message_sends.c3614068.t,$properties.$mp_message_sends.c3627207.c,$properties.$mp_message_sends.c3627207.t,$properties.$mp_message_sends.c3787003.c,$properties.$mp_message_sends.c3787003.t,$properties.$mp_message_sends.c3787088.c,$properties.$mp_message_sends.c3787088.t,$properties.$mp_message_sends.c3790627.c,$properties.$mp_message_sends.c3790627.t,$properties.$mp_message_sends.c3799568.c,$properties.$mp_message_sends.c3799568.t,$properties.$mp_message_sends.c3800327.c,$properties.$mp_message_sends.c3800327.t,$properties.$mp_message_sends.c3800652.c,$properties.$mp_message_sends.c3800652.t,$properties.$mp_message_sends.c3820051.c,$properties.$mp_message_sends.c3820051.t,$properties.$mp_message_sends.c3820091.c,$properties.$mp_message_sends.c3820091.t,$properties.$mp_message_sends.c3836573.c,$properties.$mp_message_sends.c3836573.t,$properties.$mp_message_sends.c3847019.c,$properties.$mp_message_sends.c3847019.t,$properties.$mp_message_sends.c3858479.c,$properties.$mp_message_sends.c3858479.t,$properties.$mp_message_sends.c3858496.c,$properties.$mp_message_sends.c3858496.t,$properties.$mp_message_sends.c3858508.c,$properties.$mp_message_sends.c3858508.t,$properties.$mp_message_sends.c3873500.c,$properties.$mp_message_sends.c3873500.t,$properties.$mp_message_sends.c3873556.c,$properties.$mp_message_sends.c3873556.t,$properties.$mp_message_sends.c3873571.c,$properties.$mp_message_sends.c3873571.t,$properties.$mp_message_sends.c3882903.c,$properties.$mp_message_sends.c3882903.t,$properties.$mp_message_sends.c3882911.c,$properties.$mp_message_sends.c3882911.t,$properties.$mp_message_sends.c3918476.c,$properties.$mp_message_sends.c3918476.t,$properties.$mp_message_sends.c3918596.c,$properties.$mp_message_sends.c3918596.t,$properties.$mp_message_sends.c3954584.c,$properties.$mp_message_sends.c3954584.t,$properties.$mp_message_sends.c3954696.c,$properties.$mp_message_sends.c3954696.t,$properties.$mp_message_sends.c3954823.c,$properties.$mp_message_sends.c3954823.t,$properties.$mp_message_sends.c3954884.c,$properties.$mp_message_sends.c3954884.t,$properties.$mp_message_sends.c3954899.c,$properties.$mp_message_sends.c3954899.t,$properties.$mp_message_sends.c3957156.c,$properties.$mp_message_sends.c3957156.t,$properties.$mp_message_sends.c3957171.c,$properties.$mp_message_sends.c3957171.t,$properties.$mp_message_sends.c3957499.c,$properties.$mp_message_sends.c3957499.t,$properties.$mp_message_sends.c3977607.c,$properties.$mp_message_sends.c3977607.t,$properties.$mp_message_sends.c3985496.c,$properties.$m

In [12]:
#create new variables with date and time of subcription end
people_df['Sub_End_Date'] = pd.to_datetime(people_df['$properties.$subscription_end_time']).dt.date
people_df['Sub_End_Time'] = pd.to_datetime(people_df['$properties.$subscription_end_time']).dt.time


In [15]:
#isolate dates to May only
people_df['Sub_End_Date'] = pd.to_datetime(people_df['Sub_End_Date']) 

may_cancellations_df = people_df.loc[(people_df['Sub_End_Date'] > '2019-5-1') & (people_df['Sub_End_Date'] <= '2019-5-31')]

#recode null values to zero
may_cancellations_df['$properties.$mp_message_sends.c3873500.c'] = may_cancellations_df['$properties.$mp_message_sends.c3873500.c'].fillna(0)

#keep only campaign columns from dataframe
may_cancellations_df=may_cancellations_df.loc[:,['$distinct_id','$properties.$mp_message_sends.c3873500.c','$properties.$mp_message_sends.c3873500.t','Sub_End_Date']]
may_cancellations_df=may_cancellations_df.rename(columns={'$distinct_id': 'properties.distinct_id'})

may_cancellations_df.head()


/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys


,properties.distinct_id,$properties.$mp_message_sends.c3873500.c,$properties.$mp_message_sends.c3873500.t,Sub_End_Date
450,426274,1.0,1.556032e+09,2019-05-25
662,389426,0.0,NaN,2019-05-17
689,494340,0.0,NaN,2019-05-08
823,545758,0.0,NaN,2019-05-28
896,575718,1.0,1.557453e+09,2019-05-08


In [28]:
#events API call
selector = '(("Resubscription" in properties["event"]) and (defined (properties["event"])))'
parameters = {'selector': selector}
m.export_events('event_export_2.txt',{'from_date':'2019-05-01','to_date':'2019-05-01'})

In [29]:
#import events data
with open("event_export_2.txt", 'r') as data:
    events = json.load(data)
## printing the json file, if needed    
print(json.dumps(events, indent=4, sort_keys=True))

#convert events data to dataframe
events_df = pd.DataFrame.from_dict(json_normalize(events), orient='columns')
events_df.head(10)


IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



,event,properties.$ae_crashed_reason,properties.$ae_iap_name,properties.$ae_iap_price,properties.$ae_iap_quantity,properties.$ae_session_length,properties.$ae_updated_version,properties.$app_build_number,properties.$app_release,properties.$app_version,properties.$app_version_string,properties.$bluetooth_enabled,properties.$bluetooth_version,properties.$brand,properties.$browser,properties.$browser_version,properties.$carrier,properties.$city,properties.$device,properties.$device_id,properties.$google_play_services,properties.$had_persisted_distinct_id,properties.$has_nfc,properties.$has_telephone,properties.$insert_id,properties.$lib_version,properties.$manufacturer,properties.$model,properties.$os,properties.$os_version,properties.$radio,properties.$region,properties.$screen_dpi,properties.$screen_height,properties.$screen_width,properties.$user_id,properties.$wifi,properties.Article Name,properties.Category Name,properties.Mailbox Name,properties.Site Name,properties.Tags,properties.URL,properties.Url,properties.User,properties.campaign_id,properties.category,properties.content,properties.content_id,properties.content_type,properties.courseType,properties.currency,properties.delivery_id,properties.distinct_id,properties.domain,properties.duration,properties.duration_seconds,properties.finished_at,properties.id,properties.is_journey_control,properties.journey_id,properties.message_id,properties.message_name,properties.message_type,properties.mp_country_code,properties.mp_lib,properties.mp_processing_time_ms,properties.pin,properties.pin_entered,properties.progress_id,properties.progress_type,properties.recipient,properties.revenue,properties.sent_to,properties.subscription_end_time,properties.subscription_id,properties.subscription_provider,properties.subscription_start_time,properties.subscription_type,properties.term,properties.time,properties.title,properties.trackId,properties.type,properties.url
0,$ae_session,NaN,NaN,NaN,NaN,14.2,NaN,87,87.0,1.2.2,1.2.2,False,ble,OnePlus,NaN,NaN,Digi.Mobil,Braşov,NaN,NaN,available,NaN,True,True,xcmhtyrboyvcmnco,5.3.0,OnePlus,ONEPLUS A6013,Android,9,NaN,Judetul Brasov,420.0,2261.0,1080.0,NaN,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,199776,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,RO,android,1556694000172,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1556668800,NaN,NaN,NaN,NaN
1,$ae_session,NaN,NaN,NaN,NaN,28,NaN,91,NaN,NaN,1.2.2,NaN,NaN,NaN,NaN,NaN,vf nl,Deventer,NaN,560B549E-5B6F-4F86-8B69-DB08113E751A,NaN,NaN,NaN,NaN,d1fb0c8fd7f39048,2.5.7,Apple,"iPhone8,1",iOS,12.2,LTE,Provincie Overijssel,NaN,667.0,375.0,417494,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,417494,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NL,swift,1556694000086,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1556668800,NaN,NaN,NaN,NaN
2,Remove Download,NaN,NaN,NaN,NaN,NaN,NaN,91,NaN,NaN,1.2.2,NaN,NaN,NaN,NaN,NaN,Telenor,Moss,NaN,E18CD6CE-C8C0-452C-91DA-E887877C9148,NaN,NaN,NaN,NaN,bce3f5f040ca3fe4,2.5.7,Apple,"iPhone8,4",iOS,12.2,LTE,Østfold,NaN,568.0,320.0,562959,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,562959,NaN,NaN,NaN,NaN,93,NaN,NaN,NaN,NaN,NaN,NO,swift,1556694053867,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1556668800,NaN,NaN,NaN,NaN
3,$ae_session,NaN,NaN,NaN,NaN,38.1,NaN,87,87.0,1.2.2,1.2.2,False,ble,HONOR,NaN,NaN,,Vaasa,NaN,NaN,available,NaN,True,True,bujejasijaqizjbz,5.3.0,HUAWEI,LLD-L31,Android,8.0.0,NaN,Ostrobothnia,480.0,2032.0,1080.0,NaN,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ab6f2d3a-6b98-4a1d-a4de-051a70848a96,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,FI,android,1556694000634,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1556668800,NaN,NaN,NaN,NaN
4,Server Progress Created,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Copenhagen,NaN,NaN,NaN,NaN,NaN,NaN,ovempbemumpitscq,2.2.0,NaN,NaN,NaN,NaN,NaN,Capital Region,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Na

In [30]:
#merge events dataframe and may cancellations data
events_df = pd.merge(events_df, may_cancellations_df,how='left', on='properties.distinct_id')



In [31]:
#explore frequency of various events
events_describe = events_df.groupby(['event']).count()
events_describe = events_describe.loc[:,'properties.$insert_id'].reset_index().sort_values('properties.$insert_id',ascending=False)
events_describe

,event,properties.$insert_id
3,$ae_session,87709
25,Media Start,59513
45,Server Progress Created,52468
23,Media Complete,50925
47,Start Download,42924
40,Remove Download,35134
4,$ae_updated,8651
24,Media Favorited,2705
48,Stop Download,2615
36,Pin Sent,2514


In [34]:
cancellation_assessment = events_df.loc[events_df['event']=='Resubscription']
cancellation_assessment = cancellation_assessment.loc[:,['$properties.$mp_message_sends.c3873500.c','$properties.$mp_message_sends.c3873500.t','Sub_End_Date']]                           
cancellation_assessment.head()

,$properties.$mp_message_sends.c3873500.c,$properties.$mp_message_sends.c3873500.t,Sub_End_Date
879,NaN,NaN,NaT
1197,NaN,NaN,NaT
6068,NaN,NaN,NaT
12630,NaN,NaN,NaT
28908,NaN,NaN,NaT


In [33]:
events_describe = events_df.groupby(['$properties.$mp_message_sends.c3873500.c']).count()
events_describe.head()

,event,properties.$ae_crashed_reason,properties.$ae_iap_name,properties.$ae_iap_price,properties.$ae_iap_quantity,properties.$ae_session_length,properties.$ae_updated_version,properties.$app_build_number,properties.$app_release,properties.$app_version,properties.$app_version_string,properties.$bluetooth_enabled,properties.$bluetooth_version,properties.$brand,properties.$browser,properties.$browser_version,properties.$carrier,properties.$city,properties.$device,properties.$device_id,properties.$google_play_services,properties.$had_persisted_distinct_id,properties.$has_nfc,properties.$has_telephone,properties.$insert_id,properties.$lib_version,properties.$manufacturer,properties.$model,properties.$os,properties.$os_version,properties.$radio,properties.$region,properties.$screen_dpi,properties.$screen_height,properties.$screen_width,properties.$user_id,properties.$wifi,properties.Article Name,properties.Category Name,properties.Mailbox Name,properties.Site Name,properties.Tags,properties.URL,properties.Url,properties.User,properties.campaign_id,properties.category,properties.content,properties.content_id,properties.content_type,properties.courseType,properties.currency,properties.delivery_id,properties.distinct_id,properties.domain,properties.duration,properties.duration_seconds,properties.finished_at,properties.id,properties.is_journey_control,properties.journey_id,properties.message_id,properties.message_name,properties.message_type,properties.mp_country_code,properties.mp_lib,properties.mp_processing_time_ms,properties.pin,properties.pin_entered,properties.progress_id,properties.progress_type,properties.recipient,properties.revenue,properties.sent_to,properties.subscription_end_time,properties.subscription_id,properties.subscription_provider,properties.subscription_start_time,properties.subscription_type,properties.term,properties.time,properties.title,properties.trackId,properties.type,properties.url,$properties.$mp_message_sends.c3873500.t,Sub_End_Date
$properties.$mp_message_sends.c3873500.c,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0.0,1551,22,0,0,0,379,30,1274,622,622,1274,622,622,622,0,0,1274,1438,0,652,622,4,622,622,1551,1525,1274,1274,1274,1274,652,1475,622,1274,1274,652,1270,0,0,0,0,0,0,0,0,26,0,2,2,2,549,0,17,1551,0,1,0,0,381,17,26,17,17,17,1507,1525,1551,4,3,0,0,0,0,17,11,11,11,11,11,0,1551,678,434,9,164,0,1551
1.0,1350,0,18,18,18,291,41,1155,105,105,1155,105,105,105,0,0,1155,1311,0,1050,105,35,105,105,1350,1350,1155,1155,1155,1155,1050,1324,105,1155,1155,1050,1140,0,0,0,0,0,0,0,0,0,0,0,0,0,381,15,0,1350,0,11,0,0,403,0,0,0,0,0,1345,1350,1350,3,2,0,0,0,15,0,0,0,0,0,0,0,1350,550,357,0,188,1350,1350
